In [1]:
import os
import json

import os.path as osp
os.chdir(osp.dirname(os.getcwd()))

In [5]:

import pandas as pd 

from collections import Counter

import numpy as np

from loguru import logger
'''
description:  获取SR3D 作者划分好的 训练集 和测试集
param {*} split
return {*}
'''
def get_split_list(dataset_name="sr3d" ,split='train'):
    with open('data/meta_data/%s_%s_scans.txt' % (dataset_name,split)) as f:
        scan_ids = set(eval(f.read()))
    logger.info(f" len of {dataset_name,split} : {len(scan_ids)}")
    return scan_ids


def get_ratio_split_list(ratio,dataset_name="sr3d" ,split='train'):
    with open('data/meta_data/%s_%s_%.1f.txt' % (dataset_name,split,ratio)) as f:
        scan_ids = f.read().split("\n")
    logger.info(f" len of {dataset_name,split,ratio} : {len(scan_ids)}")
    return scan_ids


def read_txt(file_name):
    with open(file_name,'r') as f:
        scan_ids = f.read().split("\n")
    logger.info(f" len of {file_name} : {len(scan_ids)}")
    return scan_ids


'''
description:  从SR3D作者那获取 详细的标注数据
param {*} split_name
return {*}
'''
def get_refer_it_3D(split_name='sr3d'):
    scanrefer_root="datasets/refer_it_3d"
    data = pd.read_csv(osp.join(scanrefer_root,split_name+".csv"))
    
    # logger.info(f"len of {split_name} : {data.shape[0]}")
    all_attrs = data.columns
    # logger.info(f" column of {split_name} : {all_attrs}")
    # logger.info(f"scene number : {len(set(data['scan_id']))}")

    stat = Counter(data['scan_id'])

    scane_stat = np.array([v for k ,v in stat.items()])
    avg_sample =scane_stat.mean()
    min_sample =scane_stat.min()
    max_sample =scane_stat.max()


    # logger.info(f"min sample: {min_sample} \n max sample : {max_sample} \n avg sample: {avg_sample}")    
    # print(data.iloc[0,:])
    return data
    
      



def generate_NR3D_labeled_scene_txt(labeled_ratio):
    nr3d_ids = get_split_list(dataset_name='nr3d')

    num_scans = len(nr3d_ids)
    num_labeled_scans = int(num_scans*labeled_ratio)


    choices = np.random.choice(num_scans, num_labeled_scans, replace=False)#* 从num_scans 挑选num_labeled_scans 个场景 出来 

    labeled_scan_names = list(np.array(list(nr3d_ids))[choices])
    
    with open(os.path.join('data/meta_data/nr3d_train_{}.txt'.format(labeled_ratio)), 'w') as f:
        f.write('\n'.join(labeled_scan_names))
    
    logger.info('\tSelected {} labeled scans, remained {} unlabeled scans'.format(len(labeled_scan_names),num_scans- len(labeled_scan_names)))


    
    
# sr3d_ids = get_split_list()
# sr3d_ids_test = get_split_list(split = 'test')
# nr3d_ids = get_split_list(dataset_name='nr3d')
# nr3d_ids_test = get_split_list(dataset_name='nr3d',split = 'test')



# get_refer_it_3D('sr3d+')
# get_refer_it_3D('sr3d')
# get_refer_it_3D('sr3d_test')
# get_refer_it_3D('sr3d_train')


2022-10-14 22:24:58.673 | INFO     | __main__:get_refer_it_3D:32 - len of sr3d_test : 17726
2022-10-14 22:24:58.674 | INFO     | __main__:get_refer_it_3D:34 -  column of sr3d_test : Index(['scan_id', 'target_id', 'distractor_ids', 'utterance', 'stimulus_id',
       'coarse_reference_type', 'reference_type', 'instance_type',
       'anchors_types', 'anchor_ids', 'tokens', 'dataset',
       'mentions_target_class', 'correct_guess', 'is_train'],
      dtype='object')
2022-10-14 22:24:58.676 | INFO     | __main__:get_refer_it_3D:35 - scene number : 255
2022-10-14 22:24:58.679 | INFO     | __main__:get_refer_it_3D:45 - min sample: 4 
 max sample : 682 
 avg sample: 69.51372549019608


scan_id                                                       scene0629_00
target_id                                                                0
distractor_ids                                                         [2]
utterance                              the door that is near the end table
stimulus_id                                        scene0629_00-door-2-0-2
coarse_reference_type                                           horizontal
reference_type                                                     closest
instance_type                                                         door
anchors_types                                                ['end table']
anchor_ids                                                             [7]
tokens                   ['the', 'door', 'that', 'is', 'near', 'the', '...
dataset                                                               sr3d
mentions_target_class                                                 True
correct_guess            

In [23]:

def statstic_nr3d():
    nr3d = get_refer_it_3D('nr3d')
    nr3d_train =  get_split_list(dataset_name='nr3d')
    len_test= len_train= 0
    for idx in range(nr3d.shape[0]):
        if nr3d.iloc[idx,:]['scan_id'] in nr3d_train:
            len_train+=1
        else:
            len_test+=1
    print(f"len of train : {len_train}, len of test : {len_test}")



def statstic_nr3d_by_ratio(ratio,datasets='nr3d'):
    
    nr3d = get_refer_it_3D(datasets)
    nr3d_ratio=  get_ratio_split_list(ratio,dataset_name=datasets)
    
    num = 0 
    for idx in range(nr3d.shape[0]):
        if nr3d.iloc[idx,:]['scan_id'] in nr3d_ratio:
            num+=1
        
    print(f"len of {datasets,ratio} : {num}")


# datasets = 'sr3d'
# for n in np.linspace(0.1,0.9,9):
#     # print(round(n,1))
#     statstic_nr3d_by_ratio(round(n,1),datasets)



FileNotFoundError: [Errno 2] No such file or directory: 'datasets/scanrefer/ScanRefer_filtered_test.json'

In [31]:
# for x in np.linspace(0.1,0.9,9):
#     generate_NR3D_labeled_scene_txt(round(x,1))


In [30]:

def get_scanrefer(split=None):
    if split is not None :
        path = "datasets/scanrefer/ScanRefer_filtered_%s.json"%split
    else :
        path = "datasets/scanrefer/ScanRefer_filtered.json"

    with open (path,'r')as f :
        data =json.load(f)
    
    length = len(data)
    # logger.info(f" len of {split} split : {length}")
    # all_scene = set([x['scene_id']  for x in data])
    # logger.info(f" scene number  of {split} split : {len(all_scene)}")

    # all_object_id = set([x['object_id']  for x in data])
    # logger.info(f" object number  of {split} split : {len(all_object_id)}")

    # all_anno_id = set([x['ann_id']  for x in data])
    # logger.info(f" anno number  of {split} split : {len(all_anno_id)}")

    # print(data[0])
    
    return data

def get_ratio_scanrefer(ratio,split=None):
    
    path = "datasets/scanrefer/ScanRefer_filtered_train_%.1f.txt"%(ratio)

    scanrefer = get_scanrefer(split='train')
    scan_ids = read_txt(path)

    
    num = 0 
    for refer in scanrefer:
        if refer['scene_id'] in scan_ids:
            num+=1
    
    print(f"len of scanrefer {ratio} : {num}")
    



def generate_scanrefer_labeled_scene_txt(labeled_ratio):
    all_scenes = get_scanrefer(split='train')

    num_scans = len(all_scenes)
    num_labeled_scans = int(num_scans*labeled_ratio)


    choices = np.random.choice(num_scans, num_labeled_scans, replace=False)#* 从num_scans 挑选num_labeled_scans 个场景 出来 

    labeled_scan_names = list(np.array(list(all_scenes))[choices])
    
    with open(os.path.join('datasets/scanrefer/ScanRefer_filtered_train_{}.txt'.format(labeled_ratio)), 'w') as f:
        f.write('\n'.join(labeled_scan_names))
    
    logger.info('\tSelected {} labeled scans, remained {} unlabeled scans'.format(len(labeled_scan_names),num_scans- len(labeled_scan_names)))


    
    
        


# get_scanrefer(split='val')
# get_scanrefer(split='test')
# get_scanrefer()
# scanrefer = get_scanrefer(split='train')




for n in np.linspace(0.1,0.9,9):
    # print(round(n,1))
    get_ratio_scanrefer(round(n,1))



2022-10-14 23:30:07.210 | INFO     | __main__:read_txt:30 -  len of datasets/scanrefer/ScanRefer_filtered_train_0.1.txt : 56
2022-10-14 23:30:07.451 | INFO     | __main__:read_txt:30 -  len of datasets/scanrefer/ScanRefer_filtered_train_0.2.txt : 112


len of scanrefer 0.1 : 3843


2022-10-14 23:30:07.698 | INFO     | __main__:read_txt:30 -  len of datasets/scanrefer/ScanRefer_filtered_train_0.3.txt : 168


len of scanrefer 0.2 : 7191


2022-10-14 23:30:07.972 | INFO     | __main__:read_txt:30 -  len of datasets/scanrefer/ScanRefer_filtered_train_0.4.txt : 224


...

len of scanrefer 0.6 : 21859


2022-10-14 23:30:09.091 | INFO     | __main__:read_txt:30 -  len of datasets/scanrefer/ScanRefer_filtered_train_0.8.txt : 449


len of scanrefer 0.7 : 25478


2022-10-14 23:30:09.416 | INFO     | __main__:read_txt:30 -  len of datasets/scanrefer/ScanRefer_filtered_train_0.9.txt : 505


len of scanrefer 0.8 : 29263
len of scanrefer 0.9 : 32832


In [5]:

# for x in np.linspace(0.1,0.9,9):
#     generate_scanrefer_labeled_scene_txt(round(x,1))

In [26]:
scanrefer = get_scanrefer(split='train')

2022-10-14 23:24:24.080 | INFO     | __main__:get_scanrefer:11 -  len of train split : 36665
2022-10-14 23:24:24.094 | INFO     | __main__:get_scanrefer:13 -  scene number  of train split : 562
2022-10-14 23:24:24.103 | INFO     | __main__:get_scanrefer:16 -  object number  of train split : 87
2022-10-14 23:24:24.106 | INFO     | __main__:get_scanrefer:19 -  anno number  of train split : 5


{'scene_id': 'scene0000_00', 'object_id': '39', 'object_name': 'cabinet', 'ann_id': '1', 'description': 'a white cabinet in the corner of the room. in the direction from the door and from the inside . it will be on the left, there is a small brown table on the left side of the cabinet and a smaller table on the right side of the cabinet', 'token': ['a', 'white', 'cabinet', 'in', 'the', 'corner', 'of', 'the', 'room', '.', 'in', 'the', 'direction', 'from', 'the', 'door', 'and', 'from', 'the', 'inside', '.', 'it', 'will', 'be', 'on', 'the', 'left', ',', 'there', 'is', 'a', 'small', 'brown', 'table', 'on', 'the', 'left', 'side', 'of', 'the', 'cabinet', 'and', 'a', 'smaller', 'table', 'on', 'the', 'right', 'side', 'of', 'the', 'cabinet']}
